In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
os.chdir('/content/gdrive/MyDrive/projectdata')

In [3]:
data_file = "./photorealistic-blocksworld-master/coordinates.pickle"

In [4]:
import numpy as np
import tensorflow as tf

In [5]:
import pickle
with open(data_file, 'rb') as f:
  data = pickle.load(f)

{'/Users/pippala/Desktop/robotic/location-based-generative-master/photorealistic-blocksworld-master/blocks-5-3/scene/CLEVR_new_000000.json': (tensor([[[0.5454, 0.4738],
           [0.6215, 0.6539],
           [0.5831, 0.7506],
           [0.6579, 0.9072],
           [1.0722, 0.7108]],
  
          [[0.6376, 0.3811],
           [0.7501, 0.6440],
           [0.6611, 0.7855],
           [0.7566, 0.9314],
           [0.8718, 0.8160]],
  
          [[0.5742, 0.3959],
           [0.6672, 0.6431],
           [0.5919, 0.7467],
           [0.6225, 0.9376],
           [1.0492, 0.7490]]]),
  tensor([[[-3.0000, -2.0000,  0.9000],
           [-3.0000, -2.0000,  2.4364],
           [-3.0000, -2.0000,  3.7092],
           [-3.0000, -2.0000,  4.9820],
           [-3.0000, -2.0000,  6.2548]],
  
          [[-3.0000, -2.0000,  0.9000],
           [-3.0000, -2.0000,  2.4364],
           [-3.0000, -2.0000,  3.7092],
           [-3.0000, -2.0000,  4.9820],
           [-3.0000, -2.0000,  6.2548]],
  
      

In [6]:
all_camera_coord = []
real_coordinates = []
for key, value in data.items():
    # print(key, value)
    image_coords = value[0].numpy()
    # print(image_coords.shape)
    real_coords = value[1].numpy()
    for i in range(5):
      all_camera_coord.append(image_coords[:, i, :])
      real_coordinates.append(real_coords[0, i, :])

In [7]:
all_camera_coord = np.array(all_camera_coord)
real_coordinates = np.array(real_coordinates)

In [8]:
real_coordinates

array([[-3.       , -2.       ,  0.9      ],
       [-3.       , -2.       ,  2.4363961],
       [-3.       , -2.       ,  3.7091882],
       ...,
       [ 3.       , -2.       ,  1.9091883],
       [ 0.       , -2.       ,  0.6363961],
       [ 3.       , -2.       ,  0.6363961]], dtype=float32)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_camera_coord, real_coordinates, test_size=0.5, random_state=42)

In [ ]:
X_train.shape 

(4882, 3, 2)

In [ ]:
X_train[0]

array([[0.9410324, 0.8222167],
       [0.7850928, 0.8210632],
       [0.8642629, 0.8085441]], dtype=float32)

(4882,)


array([0., 1., 2.], dtype=float32)

In [41]:
y_train_x = y_train[:, 0]
y_train_xp = (y_train_x + 3)/3
categorical_data_x = tf.keras.utils.to_categorical(y_train_xp, num_classes=3)

y_train_y = y_train[:, 1]
y_train_yp = (y_train_y + 2)/4
print(np.unique(y_train_yp))
print(y_train_yp)
categorical_data_y = tf.keras.utils.to_categorical(y_train_yp, num_classes=2)
print(categorical_data_y)

[0. 1.]
[1. 1. 0. ... 0. 0. 0.]
[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [ ]:
model_x = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(3,2)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model_x.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

# Train the model
model_x.fit(X_train, categorical_data_x, epochs=50, batch_size=32)

In [49]:
y_pred_cat_x = model_x.predict(X_test)
# print(np.round(y_pred_cat_x))
y_numeric_labels = np.argmax(y_pred_cat_x, axis=1)
y_pred_num = (y_numeric_labels*3)-3

loss_round = tf.reduce_mean(tf.square(y_test[:, 0] - y_pred_num))

# Print the predicted values and evaluation metrics
print("\nEvaluation round loss:", loss_round.numpy())

153/153 [==============================] - 0s 2ms/step

Evaluation round loss: 0.06819578128199877


In [51]:
model_y = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(3,2)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model
model_y.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

# Train the model
model_y.fit(X_train, categorical_data_y, epochs=50, batch_size=32)

153/153 [==============================] - 0s 2ms/step - loss: 0.0109 - accuracy: 0.9982
Epoch 45/50
153/153 [==============================] - 0s 2ms/step - loss: 0.0108 - accuracy: 0.9982
Epoch 46/50
153/153 [==============================] - 0s 2ms/step - loss: 0.0109 - accuracy: 0.9982
Epoch 47/50
153/153 [==============================] - 0s 2ms/step - loss: 0.0107 - accuracy: 0.9982
Epoch 48/50
153/153 [==============================] - 0s 2ms/step - loss: 0.0106 - accuracy: 0.9982
Epoch 49/50
153/153 [==============================] - 0s 2ms/step - loss: 0.0108 - accuracy: 0.9982
Epoch 50/50
153/153 [==============================] - 0s 2ms/step - loss: 0.0107 - accuracy: 0.9982


In [53]:
y_pred_cat_y = model_y.predict(X_test)
# print(np.round(y_pred_cat_x))
y_numeric_labels_y = np.argmax(y_pred_cat_y, axis=1)
y_pred_num_y = (y_numeric_labels_y*4)-2

loss_round = tf.reduce_mean(tf.square(y_test[:, 1] - y_pred_num_y))

# Print the predicted values and evaluation metrics
print("\nEvaluation round loss:", loss_round.numpy())

153/153 [==============================] - 0s 1ms/step

Evaluation round loss: 0.029490067581404874


In [ ]:
model_z = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(3,2)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1)
])

# Compile the model
model_z.compile(optimizer='adam', loss='mse')

# Train the model
model_z.fit(X_train, y_train[:, 2], epochs=100, batch_size=32)

In [ ]:
y_pred_z = model_z.predict(X_test)
loss_round = tf.reduce_mean(tf.square(y_test[:, 1] - y_pred_num_x))
print("\nEvaluation round loss:", loss_round.numpy())

306/306 [==============================] - 0s 1ms/step


In [64]:
y_pred_cat_x = model_x.predict(all_camera_coord)
y_numeric_labels = np.argmax(y_pred_cat_x, axis=1)
y_pred_x = (y_numeric_labels*3)-3
y_pred_cat_y = model_y.predict(all_camera_coord)
y_numeric_labels_y = np.argmax(y_pred_cat_y, axis=1)
y_pred_y = (y_numeric_labels_y*4)-2
y_pred_z = model_z.predict(all_camera_coord)

306/306 [==============================] - 0s 1ms/step


In [70]:
print(y_pred_x.shape, y_pred_y.shape, y_pred_z.shape)
print(y_pred_x.shape == y_pred_y.shape)
print(y_pred_y.shape == y_pred_z.shape)
preds = np.stack((y_pred_x.reshape(9765,1), y_pred_y.reshape(9765,1), y_pred_z), axis=2)
print(preds)

(9765,) (9765,) (9765, 1)
True
False
[[[-3.         -2.          0.80418688]]

 [[-3.         -2.          1.53947651]]

 [[-3.         -2.          2.69016409]]

 ...

 [[ 3.         -2.          2.05028009]]

 [[ 0.         -2.          0.50497466]]

 [[ 3.         -2.          0.71412718]]]


In [71]:
loss_round_total = tf.reduce_mean(tf.square(real_coordinates - preds))


In [72]:
print(loss_round_total)

tf.Tensor(7.202388279026319, shape=(), dtype=float64)


In [73]:
preds.shape

(9765, 1, 3)

In [74]:
out_preds_5obj = preds.reshape(1953, 5, 3)

In [75]:
loss_round_total = tf.reduce_mean(tf.square(real_coordinates - preds))
print(loss_round_total)

tf.Tensor(7.202388279026319, shape=(), dtype=float64)


In [76]:
out_dict = {}

In [77]:
itter = 0
for key, value in data.items():
  out_dict[key] = out_preds_5obj[itter, :, :]
  itter += 1

want as dict same dict but value not tuple but 3d coord 1953 values

In [78]:
out_dict

{'/Users/pippala/Desktop/robotic/location-based-generative-master/photorealistic-blocksworld-master/blocks-5-3/scene/CLEVR_new_000000.json': array([[-3.        , -2.        ,  0.80418688],
        [-3.        , -2.        ,  1.53947651],
        [-3.        , -2.        ,  2.69016409],
        [-3.        , -2.        ,  4.08313227],
        [ 0.        ,  2.        ,  1.81109655]]),
 '/Users/pippala/Desktop/robotic/location-based-generative-master/photorealistic-blocksworld-master/blocks-5-3/scene/CLEVR_new_000001.json': array([[-3.        , -2.        ,  0.70301145],
        [-3.        , -2.        ,  1.85171378],
        [-3.        , -2.        ,  3.30633187],
        [-3.        , -2.        ,  4.85491419],
        [ 0.        , -2.        ,  0.14547578]]),
 '/Users/pippala/Desktop/robotic/location-based-generative-master/photorealistic-blocksworld-master/blocks-5-3/scene/CLEVR_new_000002.json': array([[-3.        , -2.        ,  0.72693396],
        [-3.        , -2.        ,  1

In [79]:
with open("predicted_3D_coords_seperate", 'wb') as f:
  pickle.dump(out_dict, f)

In [80]:
ls


 analyze_random_loc.py             models.py
 blender/                          notebooks/
 calibration.py                    photorealistic-blocksworld-master/
 ccl/                              planner.py
 commands.txt                      planner_rs.py
 current_plan_with_heuristics.py   plan_with_3d_rel.py
 data/                             plan_with_heuristic.py
 data_loader.py                    predicted_3D_coords
'demo '/                           predicted_3D_coords_seperate
 edit_image.py                     pre_models/
 edit_video.py                     __pycache__/
 figures/                          README.md
 fine_scale.py                     resnet.py
 fp_callibration_auto.ipynb        sim_data/
 fp_callibration.ipynb             train_shape_predictor.py
 fp_collab.ipynb                   train_stability_predictor.py
 large_scale_pbw.py                train_uc.py
 large_scale.py                    train_uc_sim.py
 logs/                             train_uc_trivial_obj.py
 